In [10]:
import json  # Import the json module to work with JSON data
import sqlite3  # Import sqlite3 module to interact with the SQLite database

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('rosterdb.sqlite')
cur = conn.cursor()  # Create a cursor object to execute SQL commands

# Execute a block of SQL statements to set up the database schema
cur.executescript('''
DROP TABLE IF EXISTS User;           -- Drop the User table if it exists
DROP TABLE IF EXISTS Member;         -- Drop the Member table if it exists
DROP TABLE IF EXISTS Course;         -- Drop the Course table if it exists
DROP TABLE IF EXISTS UserCourseRole; -- Drop the new UserCourseRole table if it exists

-- Create the User table with an id and a unique name
CREATE TABLE User (
    id     INTEGER PRIMARY KEY,  -- id is the primary key (auto-incremented)
    name   TEXT UNIQUE           -- name is unique for each user
);

-- Create the Course table with an id and a unique title
CREATE TABLE Course (
    id     INTEGER PRIMARY KEY,  -- id is the primary key (auto-incremented)
    title  TEXT UNIQUE           -- title is unique for each course
);

-- Create the Member table to store relationships between users and courses, with a role
CREATE TABLE Member (
    user_id     INTEGER,         -- user_id refers to the user in the User table
    course_id   INTEGER,         -- course_id refers to the course in the Course table
    role        INTEGER,         -- role is the role of the user in the course (0 or 1)
    PRIMARY KEY (user_id, course_id) -- The combination of user_id and course_id is unique
);

-- Create a UserCourseRole table to store the final joined results
CREATE TABLE UserCourseRole (
    user_name TEXT,      -- user_name is the name of the user
    course_title TEXT,   -- course_title is the title of the course
    role INTEGER         -- role is the role of the user in the course (0 or 1)
);
''')

# Prompt the user to input the filename containing the JSON data
fname = input('Enter file name: ')
if len(fname) < 1:  # If no filename is provided, use the default 'roster_data.json'
    fname = 'roster_data.json'

# Open and read the JSON data from the file
str_data = open(fname).read()  # Open the file and read its contents
json_data = json.loads(str_data)  # Parse the JSON data into a Python list

# Iterate over each entry in the JSON data
for entry in json_data:
    name = entry[0]  # Extract the name from the entry
    title = entry[1]  # Extract the course title from the entry
    role = entry[2]  # Extract the role from the entry

    # Insert the user name into the User table (only if it doesn't already exist)
    cur.execute('''INSERT OR IGNORE INTO User (name)
        VALUES ( ? )''', (name, ))

    # Retrieve the user_id for the user just inserted (or already exists)
    cur.execute('SELECT id FROM User WHERE name = ? ', (name, ))
    user_id = cur.fetchone()[0]  # Fetch the user_id from the result

    # Insert the course title into the Course table (only if it doesn't already exist)
    cur.execute('''INSERT OR IGNORE INTO Course (title)
        VALUES ( ? )''', (title, ))

    # Retrieve the course_id for the course just inserted (or already exists)
    cur.execute('SELECT id FROM Course WHERE title = ? ', (title, ))
    course_id = cur.fetchone()[0]  # Fetch the course_id from the result

    # Insert or replace the relationship between the user and course in the Member table
    cur.execute('''INSERT OR REPLACE INTO Member
        (user_id, course_id, role) VALUES ( ?, ?, ? )''',
        (user_id, course_id, role))  # Insert the user_id, course_id, and role

# Join the User, Course, and Member tables to get the desired data
cur.execute('''
SELECT User.name, Course.title, Member.role
FROM Member
JOIN User ON Member.user_id = User.id  -- Join Member with User on user_id
JOIN Course ON Member.course_id = Course.id  -- Join Member with Course on course_id
''')

# Fetch all the results of the query
results = cur.fetchall()

# Iterate over the results and insert them into the UserCourseRole table
for row in results:
    cur.execute('''
    INSERT INTO UserCourseRole (user_name, course_title, role)
    VALUES (?, ?, ?)  -- Insert the user_name, course_title, and role
    ''', (row[0], row[1], row[2]))  # Use placeholders to prevent SQL injection

# Commit the transaction to save all changes to the database
conn.commit()



Enter file name:  
